In [1]:
# from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

# from typing import List
import torch
# from torch import nn
# from pathlib import Path
# import json

from embedding.model import EmbeddingModel

In [2]:
model_name_or_path = "model/hfl/chinese-roberta-wwm-ext"  

em = EmbeddingModel(
    model_name_or_path=model_name_or_path,
    device="cuda:0",
    input_feature=768,
    output_feature=128,
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
text = ["你是谁", "我爱你中国"]
tokenizer_output = tokenizer(
    text,
    padding=True,
    truncation=True,
    # max_length=max_len,
    return_tensors="pt",
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [4]:
em(**tokenizer_output)[:, :3]  # .shape

tensor([[-0.6565,  0.0123, -0.5139],
        [-0.6287,  0.1873, -0.5841]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [5]:
em.save_pretrained("modelfile/test002")
tokenizer.save_pretrained("modelfile/test002")

[2024-06-30 17:46:40,671] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


('modelfile/test002/tokenizer_config.json',
 'modelfile/test002/special_tokens_map.json',
 'modelfile/test002/vocab.txt',
 'modelfile/test002/added_tokens.json',
 'modelfile/test002/tokenizer.json')

In [6]:

em2 = EmbeddingModel("modelfile/test002", device="cuda:0")

In [7]:
em2(**tokenizer_output)[:, :3].cpu().detach().numpy().shape

(2, 3)

In [8]:
torch.allclose(em2(**tokenizer_output)[:, :3], em(**tokenizer_output)[:, :3], atol=1e-5)

True

In [9]:
tokenizer_output

{'input_ids': tensor([[ 101,  872, 3221, 6443,  102,    0,    0],
        [ 101, 2769, 4263,  872,  704, 1744,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}

In [11]:
# em2(**{"input_ids": tokenizer_output.get("input_ids")}).shape

In [17]:
# em2(**{"input_ids": tokenizer_output.get("input_ids")})